In [2]:
import pandas as pd
import numpy as np
import os
from catboost import CatBoostClassifier
from encoding_function import low_frequency_to_others
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings('ignore')

### 데이터 불러오기
- 학습: 2023년도 이전
- 검증: 2023년
- 테스트: 2024년

In [3]:
def add_y(df):
    df['rank'] = df['rank'].replace(0, 6)

    target_cols = [
        '단승', '복승', '삼복승'
    ]
    for i, col in enumerate(target_cols):
        condition_target = df['rank'] <= i+1
        df.loc[condition_target, col] = 1

    df[target_cols] = df[target_cols].fillna(0)
    df.drop(['rank'], axis=1, inplace=True)
    return df


ROOT_DIR = "data"
RANDOM_STATE = 42

train = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
test = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))

train['성별'] = train['성별'].map({'남': 0, '여': 1})
test['성별'] = test['성별'].map({'남': 0, '여': 1})

train = add_y(train)
test = add_y(test)

print(train.shape, test.shape)
train.head()

(79950, 45) (6102, 45)


,Race_ID,연도,회차,일차,경주번호,번호,등급,기수,선수명,성별,...,모터_연대율3,전탑승선수1,전탑승선수2,보트번호,보트_평균착순점,보트_연대율,특이사항,단승,복승,삼복승
0,2013_1_1_1,2013,1,1,1,1,A2,2,이용세,0,...,46.3,김신오/25,김선필/25,10,5.60,32.8,NaN,0.0,1.0,1.0
1,2013_1_1_1,2013,1,1,1,2,B1,7,손동민,0,...,70.9,김승택/3,경상수/1316,68,5.59,40.7,NaN,0.0,0.0,1.0
2,2013_1_1_1,2013,1,1,1,3,B2,11,박진서,0,...,55.7,송종해/45,문주엽/4,22,5.07,30.7,NaN,0.0,0.0,0.0
3,2013_1_1_1,2013,1,1,1,4,B1,4,경상수,0,...,75.0,이종한/4,문주엽/223,77,5.49,39.7,NaN,1.0,1.0,1.0
4,2013_1_1_1,2013,1,1,1,5,B1,8,정주현,0,...,22.4,이창규/6,반혜진/545,18,5.69,37.6,NaN,0.0,0.0,0.0


In [4]:
def add_weight_penalty(df):
    weight_limit_male = 55
    weight_limit_female = 51
    df['체중'] = df['체중'].astype(float)

    # 중량 부과 여부 계산 (성별에 따라 다른 기준 적용)
    df['중량부과여부'] = df.apply(lambda row: 1 if (row['성별'] == 0 and row['체중'] < weight_limit_male) or
                                             (row['성별'] == 1 and row['체중'] < weight_limit_female) else 0, axis=1)

    # 부과된 중량 계산 (성별에 따라 차감된 체중을 계산)
    df['부과된중량'] = df.apply(lambda row: max(0, weight_limit_male - row['체중']) if row['성별'] == 0 else
    max(0, weight_limit_female - row['체중']), axis=1)

    df.drop('체중', axis=1, inplace=True)

    return df

train = add_weight_penalty(train)
test = add_weight_penalty(test)

In [5]:
def bin_age(df):
    # 나이를 5개 구간으로 나누어 bin화 (예: 0-20, 20-30, 30-40, 40-50, 50+)
    bins = [20, 30, 40, 50, 100]
    labels = ['20-30', '30-40', '40-50', '50+']

    # 나이 변수를 bin화
    df['나이'] = pd.cut(df['나이'], bins=bins, labels=labels, right=False)

    return df

train = bin_age(train)
test = bin_age(test)

## 사용하지 않는 feature drop

- 전일 성적을 어떻게 반영할지

In [6]:
def drop_columns_from_datasets(df):
    drop_cols = [
        '연도', '회차', '일차', '경주번호',
        '금일출주경주',
        '모터번호', '전탑승선수1', '전탑승선수2',
        '보트번호', '특이사항',
        'FL' # F뒤의 숫자는 현재 반기의 실격 횟수, L뒤의 숫자는 반칙횟수?
    ]

    df = df.drop(drop_cols, axis=1)

    return df


train = drop_columns_from_datasets(train)
test = drop_columns_from_datasets(test)

print(train.shape, test.shape)

(79950, 35) (6102, 35)


In [7]:
def reverse_rank_values(df_train, df_val):
    cols_to_reverse = [
        '최근6회차_평균착순점', '최근6회차_평균득점',
        '연간성적_평균착순점',
        '모터_평균착순점',
        '보트_평균착순점'
    ]
    for col in cols_to_reverse:
        max_rank = df_train[col].max()
        # print(f'{col}: {max_rank}')
        df_train[col] = max_rank - df_train[col]
        df_val[col] = max_rank - df_val[col]

        df_train[col].fillna(0, inplace=True)
        df_val[col].fillna(0, inplace=True)

    return df_train, df_val


train, test = reverse_rank_values(train, test)

In [8]:
import re

def extract_numbers(result):
    if isinstance(result, str):  # result가 문자열인 경우에만 처리
        return re.findall(r'-(\d+)-', result)  # -숫자- 형식에서 중간 숫자 추출
    return []  # 문자열이 아니면 빈 리스트 반환

def calculate_mean(numbers):
    numbers = [int(num) for num in numbers]  # 리스트 안의 문자열 숫자를 정수로 변환
    if len(numbers) > 0:
        return np.mean(numbers)  # 리스트가 비어있지 않으면 평균 계산
    else:
        return np.nan  # 빈 리스트인 경우 NaN 반환

def last_race_process(df_train, df_val):
    df_train['전일성적'] = df_train['전일성적'].apply(extract_numbers)  # 순위만 추출 (코스도 같이 추출?)
    df_train['전일성적'] = df_train['전일성적'].apply(calculate_mean) # 평균 계산

    df_val['전일성적'] = df_val['전일성적'].apply(extract_numbers)
    df_val['전일성적'] = df_val['전일성적'].apply(calculate_mean)

    max_rank = df_train['전일성적'].max()
    df_train['전일성적'] = max_rank - df_train['전일성적'] + 1
    df_val['전일성적'] = max_rank - df_val['전일성적'] + 1

    df_train['전일성적'].fillna(1, inplace=True)
    df_val['전일성적'].fillna(1, inplace=True)

    return df_train, df_val

train, test = last_race_process(train, test)

## 일부 숫자형 변수 변환

- 코스별 성적/경기수 분리

In [9]:
def separation_course(df):
    col_list = [
        '코스_1코스', '코스_2코스', '코스_3코스', '코스_4코스', '코스_5코스', '코스_6코스'
    ]
    for col in col_list:
        df[[f'{col[3:]}_성적', f'{col[3:]}_경기수']] = df[col].fillna('').str.split('/', expand=True)

    df.drop(col_list, axis=1, inplace=True)

    return df



train = separation_course(train)
test = separation_course(test)

print(train.shape, test.shape)

(79950, 41) (6102, 41)


- 코스별 성적 스무딩

In [10]:
def apply_laplace_smoothing(df, col, global_mean, alpha):
    # 경기수 0인 값이 너무 높게 나오는 경향이 있어 분모에 상수 1 추가(없애도 됨)
    encoded_value = (df[f'{col}_성적'] * df[f'{col}_경기수'] + global_mean * alpha) / (1 + df[f'{col}_경기수'] + alpha)
    df[f'{col}_성적'] = encoded_value

    return df

def laplace_smoothing_to_course(train, val=None, alpha=1):
    col_list = [
        '1코스', '2코스', '3코스', '4코스', '5코스', '6코스'
    ]
    for col in col_list:
        train[f'{col}_성적'] = train[f'{col}_성적'].astype(float)
        train[f'{col}_경기수'] = train[f'{col}_경기수'].astype(float)
        if val is not None:
            val[f'{col}_성적'] = val[f'{col}_성적'].astype(float)
            val[f'{col}_경기수'] = val[f'{col}_경기수'].astype(float)

    # Train 데이터에서 글로벌 평균 계산
    global_means = {col: train[f'{col}_성적'].mean() for col in col_list}

    for col in col_list:
        # Train 데이터에 라플라스 스무딩 적용
        train = apply_laplace_smoothing(train, col, global_means[col], alpha)
        train.drop(f'{col}_경기수', axis=1, inplace=True)

    if val is not None:
        for col in col_list:
            # Validation 데이터에 Train에서 구한 글로벌 평균으로 라플라스 스무딩 적용
            val = apply_laplace_smoothing(val, col, global_means[col], alpha)
            val.drop(f'{col}_경기수', axis=1, inplace=True)

    if val is not None:
        return train, val
    else:
        return train


train, test = laplace_smoothing_to_course(train, test, alpha=1) # 알파가 작을수록 빈도수에 가깝세, 알파가 클수록 전체 평균에 가깝게
print(train.shape, test.shape)

(79950, 35) (6102, 35)


In [11]:
def set_course_scores(df):
    # '코스_성적' 열을 초기화
    df['코스_성적'] = 0

    # '번호' 열에 따른 '코스_성적' 값 설정
    for i in range(1, 7):
        mask = df['번호'] == i
        df.loc[mask, '코스_성적'] = df.loc[mask, f'{i}코스_성적']

    # 제거할 열 목록
    drop_cols = [f'{i}코스_성적' for i in range(1, 7)]

    # 열 제거
    df = df.drop(columns=drop_cols)

    return df


from sklearn.decomposition import PCA
def apply_pca(df_train, df_val, n_components=2):
    course_cols = [f'{i}코스_성적' for i in range(1, 7)]

    pca = PCA(n_components=n_components)
    pca.fit(df_train[course_cols])

    # 훈련 데이터에 PCA 변환 적용 (transform)
    train_pca = pca.transform(df_train[course_cols])
    val_pca = pca.transform(df_val[course_cols])
    for i in range(n_components):
        col_name = f'PCA_코스성적_{i+1}'
        df_train[col_name] = train_pca[:, i]
        df_val[col_name] = val_pca[:, i]

    # 각 컴포넌트별 분산 설명 비율 출력
    # explained_variance_ratios = pca.explained_variance_ratio_
    # for i, ratio in enumerate(explained_variance_ratios):
    #     print(f"PCA Component {i+1}: {ratio:.4f} variance explained")

    # n_components=2가 제일 좋아보임

    return df_train, df_val


train, test = apply_pca(train, test, n_components=2)
train = set_course_scores(train)
test = set_course_scores(test)

- 최근 8경기 착순 분리

In [12]:
def split_last_eight_rank(df):
    for i in range(0, 4):
        df[f'최근{i+1}경기_착순'] = df['최근8경주_착순'].str[i]
    for j in range(5, 9):
        df[f'최근{j}경기_착순'] = df['최근8경주_착순'].str[j]

    df.drop('최근8경주_착순', axis=1, inplace=True)

    return df

# def adjust_for_top3(df):
#     col_list = [
#         '최근1경기_착순', '최근2경기_착순', '최근3경기_착순', '최근4경기_착순',
#         '최근5경기_착순', '최근6경기_착순', '최근7경기_착순', '최근8경기_착순'
#     ]
#
#     for col in col_list:
#     # 순위가 1, 2, 3이 아닌 경우, 결측인 경우, 6으로 조정
#     # (일반화된 성능을 위해 + 3등내에 드는게 중요)
#         df[col] = df[col].apply(lambda x: x if x in ['1', '2', '3'] else '-1')
#
#     return df

def adjust_last_eight_rank(df_train, df_val):
    col_list = [
        '최근1경기_착순', '최근2경기_착순', '최근3경기_착순', '최근4경기_착순',
        '최근5경기_착순', '최근6경기_착순', '최근7경기_착순', '최근8경기_착순'
    ]

    for col in col_list:
        df_train[col] = df_train[col].fillna(6).astype(int)
        df_val[col] = df_val[col].fillna(6).astype(int)

        # df_train[col] = df_train[col].replace(0, 6)  # 가끔씩 0이 있는 경우가 존재
        # df_val[col] = df_val[col].replace(0, 6)

        max_rank = df_train[col].max()
        df_train[col] = max_rank - df_train[col]
        df_val[col] = max_rank - df_val[col]

    first_mean_cols = 3
    col_to_mean_1 = col_list[:first_mean_cols]

    df_train[f'최근{first_mean_cols}경기_평균'] = df_train[col_to_mean_1].mean(axis=1)
    df_val[f'최근{first_mean_cols}경기_평균'] = df_val[col_to_mean_1].mean(axis=1)

    df_train.drop(col_list, axis=1, inplace=True)
    df_val.drop(col_list, axis=1, inplace=True)

    return df_train, df_val


train = split_last_eight_rank(train)
# train = adjust_for_top3(train)

test = split_last_eight_rank(test)
# test = adjust_for_top3(test)

train, test = adjust_last_eight_rank(train, test)

print(train.shape, test.shape)

(79950, 32) (6102, 32)


## 범주형 변수 확인


In [13]:
def cal_cat_cols(train, val=None):
    objective_cols = []

    # 특정 문자열이 포함된 열을 범주형 변수로 지정
    # cat_kerword_list = ['번호', '기수', '경기_착순']
    cat_kerword_list = ['번호']
    for col in train.columns:
        if any(sub in col for sub in cat_kerword_list):
            objective_cols.append(col)
            train[col] = train[col].astype('str')
            val[col] = val[col].astype('str')

    # 나머지 열에 대해 숫자형 변환 시도
    for col in train.columns:
        if col in objective_cols:
            continue  # 이미 범주형으로 처리된 열은 제외
        try:
            # 'float' 타입으로 변환 시도
            train[col] = train[col].astype('float')
            val[col] = val[col].astype('float')
        except:
            objective_cols.append(col)

    cat_features = list(set(objective_cols) - set(['rank', 'Race_ID']))
    cat_features = [feature for feature in cat_features if '단승' not in feature]
    cat_features = [feature for feature in cat_features if '복승' not in feature]
    cat_features = [feature for feature in cat_features if '삼복승' not in feature]

    return cat_features


cat_features = cal_cat_cols(train, test)
cat_features

['번호', '선수명', '나이', '등급']

## X, y 분리

In [14]:
drop_cols = [
    'Race_ID', '번호', '단승', '복승', '삼복승', '선수명', '기수'
]

X_train = train.drop(drop_cols, axis=1)
y_train = train[['단승']]
X_test = test.drop(drop_cols, axis=1)
y_test = test[['단승']]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(79950, 25) (79950, 1) (6102, 25) (6102, 1)


In [15]:
def reshape_race_data(df, players_per_race=6):
    # numpy 배열로 변환
    values = df.values

    # 데이터 형태 변환 (6개의 행을 1개의 행으로)
    reshaped_values = values.reshape(-1, players_per_race * values.shape[1])

    # 새로운 컬럼 이름 생성
    columns = [f'{col}_{i+1}번선수' for i in range(players_per_race) for col in df.columns]

    # 재구성된 DataFrame 생성
    reshaped_df = pd.DataFrame(reshaped_values, columns=columns)

    return reshaped_df

X_train = reshape_race_data(X_train, players_per_race=6)
y_train = reshape_race_data(y_train, players_per_race=6)
y_train = np.array(y_train).astype(int)

X_test = reshape_race_data(X_test, players_per_race=6)
y_test = reshape_race_data(y_test, players_per_race=6)
y_test = np.array(y_test).astype(int)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(13325, 150) (13325, 6) (1017, 150) (1017, 6)


In [16]:
def low_to_others(train, val, threshold=5, is_train=True, verbose=False):
    cat_features = cal_cat_cols(train, val)
    if is_train==False:
        cat_features = list(set(cat_features) - set(['Race_ID_1번선수', 'Race_ID_2번선수', 'Race_ID_3번선수', 'Race_ID_4번선수', 'Race_ID_5번선수', 'Race_ID_6번선수']))

    for col in cat_features:
        unifier = low_frequency_to_others(threshold=threshold, verbose=verbose)
        train[col] = unifier.fit_transform(train[col])
        val[col] = unifier.transform(val[col])

    return train, val

X_train, X_test = low_to_others(X_train, X_test, threshold=5, verbose=False)

In [35]:
def all_precoess(train, val, target='단승', is_train=True):
    train['성별'] = train['성별'].map({'남': 0, '여': 1})
    val['성별'] = val['성별'].map({'남': 0, '여': 1})

    train = add_y(train)
    if is_train:
        val = add_y(val)

    train = add_weight_penalty(train) # 연승 정확도를 높이려면 제거
    val = add_weight_penalty(val)

    train = bin_age(train)
    val = bin_age(val)

    train = drop_columns_from_datasets(train)
    val = drop_columns_from_datasets(val)

    train, val = last_race_process(train, val)

    train, val = reverse_rank_values(train, val)

    train = separation_course(train)
    val = separation_course(val)

    train, val = laplace_smoothing_to_course(train, val, alpha=1)

    train, val = apply_pca(train, val, n_components=2)

    train = set_course_scores(train)
    val = set_course_scores(val)

    train = split_last_eight_rank(train)
    # train = adjust_for_top3(train)

    val = split_last_eight_rank(val)
    # val = adjust_for_top3(val)

    train, val = adjust_last_eight_rank(train, val)

    if is_train:
        drop_cols = [
            'Race_ID', '번호', '단승', '복승', '삼복승', '선수명', '기수'
        ]
    else:
        drop_cols = [
            '번호', '선수명', '기수'
        ]

    X_train = train.drop(drop_cols, axis=1)
    y_train = train[[target]]
    X_train = reshape_race_data(X_train, players_per_race=6)
    y_train = reshape_race_data(y_train, players_per_race=6)
    y_train = np.array(y_train).astype(int)

    X_val = val.drop(drop_cols, axis=1)
    X_val = reshape_race_data(X_val, players_per_race=6)
    if is_train:
        y_val = val[[target]]
        y_val = reshape_race_data(y_val, players_per_race=6)
        y_val = np.array(y_val).astype(int)

    X_train, X_val = low_to_others(X_train, X_val, threshold=5, is_train=is_train, verbose=False)

    if is_train:
        return X_train, y_train, X_val, y_val
    else:
        return X_train, y_train, X_val

train = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
test = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))

X_train, y_train, X_test, y_test = all_precoess(train, test, is_train=True)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(13325, 150) (13325, 6) (1017, 150) (1017, 6)


In [36]:
X_train

,등급_1번선수,성별_1번선수,나이_1번선수,최근6회차_평균착순점_1번선수,최근6회차_평균득점_1번선수,최근6회차_승률_1번선수,최근6회차_연대율2_1번선수,최근6회차_연대율3_1번선수,최근6회차_평균ST_1번선수,연간성적_평균착순점_1번선수,...,모터_연대율2_6번선수,모터_연대율3_6번선수,보트_평균착순점_6번선수,보트_연대율_6번선수,중량부과여부_6번선수,부과된중량_6번선수,PCA_코스성적_1_6번선수,PCA_코스성적_2_6번선수,코스_성적_6번선수,최근3경기_평균_6번선수
0,A2,0.0,40-50,4.45,4.47,16.7,38.9,55.6,0.23,10.67,...,40.0,51.3,6.85,22.4,1.0,1.0,-14.587151,3.603727,3.175812,2.000000
1,B2,0.0,20-30,10.06,10.00,0.0,0.0,0.0,0.00,10.67,...,35.6,49.2,5.02,36.1,0.0,0.0,-29.755140,-2.902233,7.939530,0.000000
2,A1,0.0,30-40,2.62,2.58,44.4,55.6,66.7,0.24,10.67,...,35.8,50.6,5.74,31.1,0.0,0.0,32.831973,-5.173727,14.509883,2.333333
3,B1,0.0,40-50,6.98,7.00,0.0,0.0,15.4,0.31,10.67,...,25.4,38.0,6.12,28.6,0.0,0.0,11.367263,-3.369297,16.554152,2.000000
4,A2,0.0,30-40,5.81,6.09,6.3,18.8,37.5,0.25,10.67,...,54.3,67.9,6.49,15.8,1.0,4.0,-29.755140,-2.902233,7.939530,4.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13320,B1,0.0,30-40,4.25,4.25,31.3,50.0,56.3,0.19,5.17,...,26.9,41.9,4.87,40.7,0.0,0.0,27.592173,28.184798,14.509883,2.000000
13321,A1,0.0,50+,3.59,3.71,29.4,52.9,70.6,0.19,4.39,...,29.3,50.3,5.22,37.9,1.0,2.0,-34.947279,14.043612,2.268437,3.333333
13322,B1,0.0,30-40,5.12,5.28,16.7,22.2,44.4,0.19,5.74,...,28.9,46.0,6.08,27.6,0.0,0.0,-26.255820,-11.426710,1.984883,2.333333
13323,B1,0.0,40-50,6.43,6.75,6.3,12.5,31.3,0.32,6.31,...,30.5,45.6,5.20,37.7,0.0,0.0,15.335452,-22.696802,12.886562,3.666667


In [47]:
from model_function import evaluate_
from model_function import cal_params
from model_function import custom_CatBoostClassifier
from sklearn.metrics import f1_score, accuracy_score


def objective(trial):
    seed = 42
    target_value='복승'
    params = {
        'random_seed': seed,
        'iterations': 200,
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.0001, 10),
        'depth': trial.suggest_int('depth', 4, 7),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 0.5, 8),
        'verbose': 0,
        'thread_count': -1,
    }

    train = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
    unique_race_ids = train['Race_ID'].unique()
    kf = KFold(n_splits=5, shuffle=True, random_state=seed)

    scores = []
    for i, (train_race_ids, val_race_ids) in enumerate(kf.split(unique_race_ids)):
        train_ids = unique_race_ids[train_race_ids]
        val_ids = unique_race_ids[val_race_ids]

        train_fold = train[train['Race_ID'].isin(train_ids)].reset_index(drop=True)
        val_fold = train[train['Race_ID'].isin(val_ids)].reset_index(drop=True)

        X_train, y_train, X_val, y_val = all_precoess(train_fold, val_fold, target=target_value, is_train=True)
        label_number = 5
        y_train_i = y_train[:, label_number]
        y_val_i = y_val[:, label_number]
        
        model = CatBoostClassifier(**params)
        
        cat_features = cal_cat_cols(X_train, X_val)
        model.fit(
            X_train, y_train_i,
            cat_features=cat_features
        )

        y_pred = model.predict_proba(X_val)[:, 1]
        y_pred_class = (y_pred >= 0.5).astype(int)
 
        score = f1_score(y_pred_class, y_val_i, pos_label=1)
        
        scores.append(score)

    print('-'*50)
    print(f'folds 1:{scores[0]}, folds 2:{scores[1]}, folds 3:{scores[2]}, folds 4:{scores[3]}, folds 5:{scores[4]}')
    print(f'AVG F1: {np.mean(scores)}')
    print('-'*50)
    print('-'*50)

    return np.mean(scores)


import optuna

study = optuna.create_study(direction='maximize')

# study.optimize(objective, n_trials=100, show_progress_bar=True)
study.optimize(objective, n_trials=200)

print('Best trial:')
trial = study.best_trial
print(f'  Value: {trial.value}')
print('  Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

[I 2024-09-06 14:30:06,406] A new study created in memory with name: no-name-85727eeb-ff92-445e-ba2d-3df5582ee0fb
[I 2024-09-06 14:30:42,957] Trial 0 finished with value: 0.3263208485417798 and parameters: {'learning_rate': 0.17549663382284475, 'l2_leaf_reg': 1.9878075920293294, 'depth': 5, 'scale_pos_weight': 2.790547783389079}. Best is trial 0 with value: 0.3263208485417798.


--------------------------------------------------
folds 1:0.29161290322580646, folds 2:0.3350253807106599, folds 3:0.31970260223048325, folds 4:0.3233082706766917, folds 5:0.3619550858652576
AVG F1: 0.3263208485417798
--------------------------------------------------
--------------------------------------------------


[I 2024-09-06 14:31:32,423] Trial 1 finished with value: 0.1708286079937941 and parameters: {'learning_rate': 0.1596739792362628, 'l2_leaf_reg': 3.0609633158932192, 'depth': 7, 'scale_pos_weight': 1.227440461557269}. Best is trial 0 with value: 0.3263208485417798.


--------------------------------------------------
folds 1:0.18281535648994515, folds 2:0.15664845173041894, folds 3:0.15636363636363637, folds 4:0.16376306620209058, folds 5:0.19455252918287938
AVG F1: 0.1708286079937941
--------------------------------------------------
--------------------------------------------------


[I 2024-09-06 14:32:21,550] Trial 2 finished with value: 0.19160864713690223 and parameters: {'learning_rate': 0.18491585746790534, 'l2_leaf_reg': 9.974406673395997, 'depth': 7, 'scale_pos_weight': 1.3934939476782087}. Best is trial 0 with value: 0.3263208485417798.


...

--------------------------------------------------
folds 1:0.37833827893175076, folds 2:0.4087694483734088, folds 3:0.38591549295774646, folds 4:0.40951694304253783, folds 5:0.39651416122004357
AVG F1: 0.3958108649050975
--------------------------------------------------
--------------------------------------------------


[I 2024-09-06 17:00:09,385] Trial 198 finished with value: 0.3921109536764683 and parameters: {'learning_rate': 0.03371362703927272, 'l2_leaf_reg': 7.150559063662287, 'depth': 4, 'scale_pos_weight': 4.658476134561856}. Best is trial 31 with value: 0.4033981076060537.


--------------------------------------------------
folds 1:0.37509607993850885, folds 2:0.4172767203513909, folds 3:0.37293016558675307, folds 4:0.4100666173205033, folds 5:0.3851851851851852
AVG F1: 0.3921109536764683
--------------------------------------------------
--------------------------------------------------


[I 2024-09-06 17:00:58,113] Trial 199 finished with value: 0.3924982151244094 and parameters: {'learning_rate': 0.04378186909088786, 'l2_leaf_reg': 6.745917537379308, 'depth': 4, 'scale_pos_weight': 4.862513306301787}. Best is trial 31 with value: 0.4033981076060537.


--------------------------------------------------
folds 1:0.37104072398190047, folds 2:0.41, folds 3:0.38288920056100983, folds 4:0.41007194244604317, folds 5:0.38848920863309355
AVG F1: 0.3924982151244094
--------------------------------------------------
--------------------------------------------------
Best trial:
  Value: 0.4033981076060537
  Params: 
    learning_rate: 0.030900165600523555
    l2_leaf_reg: 7.093376467981388
    depth: 6
    scale_pos_weight: 4.638155043737476
